### 주문 패턴 기반 배치
> SKU별 단일 주문 경향 / 동시 주문 경향 / 주문 빈도 > 총 가중치 계산

1) 주문 경향 : 정규화된 공출현율

In [29]:
import pandas as pd
import numpy as np

# 데이터 로드
orders = pd.read_csv("./data/Sample_InputData.csv")  # 경로를 적절히 수정하세요
orders_clean = orders[['ORD_NO', 'SKU_CD']]

# 주문-SKU 매트릭스 생성
order_sku_matrix = orders_clean.pivot_table(index='ORD_NO', columns='SKU_CD', aggfunc=lambda x: 1, fill_value=0)

# SKU 간 동시 출현 행렬 계산
co_matrix = order_sku_matrix.T @ order_sku_matrix
np.fill_diagonal(co_matrix.values, 0)

# SKU별 빈도 및 동시 출현 합
sku_freq = order_sku_matrix.sum(axis=0)
co_sum = co_matrix.sum(axis=1)

# SKU 개수
num_skus = len(order_sku_matrix.columns)

# 동시 주문 경향: 동시 출현 평균 비율로 정규화
simultaneous_order_prop = (co_sum / sku_freq) / (num_skus - 1)
single_order_prop = 1 - simultaneous_order_prop

# 총 가중치 계산
total_weight = []
for sku in order_sku_matrix.columns:
    freq = sku_freq[sku]
    s_prop = single_order_prop[sku]
    sim_prop = simultaneous_order_prop[sku]
    weight = s_prop * freq if s_prop > sim_prop else sim_prop * freq
    total_weight.append(weight)

# 데이터프레임 구성 및 정렬
tendency_df = pd.DataFrame({
    'SKU': order_sku_matrix.columns,
    '단일주문경향': single_order_prop.values,
    '동시주문경향': simultaneous_order_prop.values,
    '주문빈도': sku_freq.values,
    '총가중치': total_weight
}).sort_values(by='총가중치', ascending=False).reset_index(drop=True)

# 결과 출력
print(tendency_df)

          SKU    단일주문경향    동시주문경향  주문빈도      총가중치
0    SKU_0117  0.990448  0.009552    10  9.904478
1    SKU_0307  0.993035  0.006965     9  8.937313
2    SKU_0255  0.992371  0.007629     9  8.931343
3    SKU_0053  0.989718  0.010282     9  8.907463
4    SKU_0180  0.992910  0.007090     8  7.943284
..        ...       ...       ...   ...       ...
331  SKU_0212  0.997015  0.002985     1  0.997015
332  SKU_0249  0.997015  0.002985     1  0.997015
333  SKU_0211  0.997015  0.002985     1  0.997015
334  SKU_0327  0.988060  0.011940     1  0.988060
335  SKU_0167  0.988060  0.011940     1  0.988060

[336 rows x 5 columns]


In [43]:
import pandas as pd

# 주문 데이터 불러오기
orders = pd.read_csv("./data/Sample_InputData.csv")  # 파일 경로를 상황에 맞게 수정하세요
orders_clean = orders[['ORD_NO', 'SKU_CD']]

# 1. 주문별 SKU 개수 계산
order_sku_counts = orders_clean.groupby('ORD_NO')['SKU_CD'].nunique()

# 2. 2개 이상 SKU가 포함된 주문만 필터링
multi_sku_orders = order_sku_counts[order_sku_counts > 1].index

# 3. SKU별 전체 주문 빈도
sku_total_orders = orders_clean.groupby('SKU_CD')['ORD_NO'].nunique()

# 4. SKU별 "다SKU 주문" 등장 횟수
sku_in_multi_orders = orders_clean[orders_clean['ORD_NO'].isin(multi_sku_orders)] \
    .groupby('SKU_CD')['ORD_NO'].nunique()

# 5. 동시 주문 경향 = 다SKU 주문 등장 비율
simultaneous_prop = sku_in_multi_orders / sku_total_orders
simultaneous_prop = simultaneous_prop.fillna(0)  # 단일 주문만 있는 SKU는 NaN → 0

# 6. 단일 주문 경향 = 1 - 동시 주문 경향
single_prop = 1 - simultaneous_prop

# 7. 총 가중치 계산
total_weight = []
for sku in sku_total_orders.index:
    freq = sku_total_orders[sku]
    s_prop = single_prop[sku]
    sim_prop = simultaneous_prop[sku]
    weight = s_prop * freq if s_prop > sim_prop else sim_prop * freq
    total_weight.append(weight)

# 8. 결과 데이터프레임 구성 및 정렬
balanced_tendency_df = pd.DataFrame({
    'SKU': sku_total_orders.index,
    '단일주문경향': single_prop.values,
    '동시주문경향': simultaneous_prop.values,
    '주문빈도': sku_total_orders.values,
    '총가중치': total_weight
}).sort_values(by='총가중치', ascending=False).reset_index(drop=True)

# 9. 결과 출력
print(balanced_tendency_df)
print(len(balanced_tendency_df[balanced_tendency_df['단일주문경향'] >= 0.5])) # 19


          SKU    단일주문경향    동시주문경향  주문빈도  총가중치
0    SKU_0117  0.000000  1.000000    10  10.0
1    SKU_0053  0.000000  1.000000     9   9.0
2    SKU_0253  0.000000  1.000000     8   8.0
3    SKU_0195  0.000000  1.000000     8   8.0
4    SKU_0307  0.111111  0.888889     9   8.0
..        ...       ...       ...   ...   ...
331  SKU_0287  1.000000  0.000000     1   1.0
332  SKU_0211  0.000000  1.000000     1   1.0
333  SKU_0064  1.000000  0.000000     1   1.0
334  SKU_0086  1.000000  0.000000     1   1.0
335  SKU_0001  0.500000  0.500000     2   1.0

[336 rows x 5 columns]
19


2) 주문 경향 : 자카드 유사도

- A와 B가 함께 주문된 주문 수 / A 또는 B가 포함된 주문 수

In [ ]:
import pandas as pd
import numpy as np
from dataclasses import dataclass
from typing import Dict, List, Set

@dataclass
class WarehouseParameters:
    picking_time: float
    walking_time: float
    cart_capacity: int
    rack_capacity: int
    number_pickers: int

class WarehouseSolver:
    def __init__(self, orders: pd.DataFrame, parameters: pd.DataFrame, od_matrix: pd.DataFrame):
        self.orders = orders.copy()
        self.params = self._load_parameters(parameters)
        self.od_matrix = od_matrix
        self.start_location = od_matrix.index[0]
        self.end_location = od_matrix.index[1]

        self._initialize_orders()
        self._validate_input()

    def _load_parameters(self, parameters: pd.DataFrame) -> WarehouseParameters:
        get_param = lambda x: parameters.loc[parameters['PARAMETERS'] == x, 'VALUE'].iloc[0]
        return WarehouseParameters(
            picking_time=float(get_param('PT')),
            walking_time=float(get_param('WT')),
            cart_capacity=int(get_param('CAPA')),
            rack_capacity=int(get_param('RK')),
            number_pickers=int(get_param('PK'))
        )

    def _initialize_orders(self) -> None:
        self.orders['LOC'] = pd.NA
        self.orders['LOC'] = self.orders['LOC'].astype(str)
        self.orders['CART_NO'] = pd.NA
        self.orders['SEQ'] = pd.NA

    def _validate_input(self) -> None:
        if self.orders.empty or self.od_matrix.empty:
            raise ValueError("Input data or OD matrix is empty")
        required_columns = {'ORD_NO', 'SKU_CD'}
        if not required_columns.issubset(self.orders.columns):
            raise ValueError(f"Missing required columns: {required_columns - set(self.orders.columns)}")

    def solve_storage_location(self) -> None:
        # 단일/동시 주문 경향 계산
        order_counts = self.orders.groupby('ORD_NO')['SKU_CD'].nunique()
        multi_orders = order_counts[order_counts > 1].index

        sku_total_orders = self.orders.groupby('SKU_CD')['ORD_NO'].nunique()
        sku_multi_orders = self.orders[self.orders['ORD_NO'].isin(multi_orders)].groupby('SKU_CD')['ORD_NO'].nunique()

        sim_prop = sku_multi_orders / sku_total_orders
        sim_prop = sim_prop.fillna(0)
        single_prop = 1 - sim_prop

        sku_freq = sku_total_orders

        weight = []
        for sku in sku_freq.index:
            f = sku_freq[sku]
            s = single_prop[sku]
            m = sim_prop[sku]
            w = s * f if s >= 0.5 else m * f
            weight.append((sku, s, m, f, w))

        df = pd.DataFrame(weight, columns=['SKU', '단일주문경향', '동시주문경향', '주문빈도', '총가중치'])
        df = df.sort_values(by='총가중치', ascending=False).reset_index(drop=True)

        # 입출고/경유지 기준 거리 정렬
        closeness_racks = self.od_matrix.loc[self.start_location].sort_values().index.tolist()
        betweenness_racks = self.od_matrix.loc[self.end_location].sort_values().index.tolist()

        all_racks = list(dict.fromkeys(closeness_racks + betweenness_racks))  # 중복 제거 유지
        sku_to_location = {}
        rack_cursor = 0

        for _, row in df.iterrows():
            sku = row['SKU']
            if row['단일주문경향'] >= 0.5:
                # 단일 주문 경향이 0.5 이상이면 입출고지 기준 가까운 랙 배치
                for loc in closeness_racks:
                    if list(sku_to_location.values()).count(loc) < self.params.rack_capacity:
                        sku_to_location[sku] = loc
                        break
            else:
                # 동시 주문 경향이 더 크면 경유지 기준 가까운 랙 배치
                for loc in betweenness_racks:
                    if list(sku_to_location.values()).count(loc) < self.params.rack_capacity:
                        sku_to_location[sku] = loc
                        break

        self.orders['LOC'] = self.orders['SKU_CD'].map(sku_to_location)

    def solve_order_batching(self) -> None:
        unique_orders = sorted(self.orders['ORD_NO'].unique())
        num_carts = len(unique_orders) // self.params.cart_capacity + 1

        order_to_cart = {}
        for cart_no in range(1, num_carts + 1):
            start_idx = (cart_no - 1) * self.params.cart_capacity
            end_idx = start_idx + self.params.cart_capacity
            cart_orders = unique_orders[start_idx:end_idx]
            for order in cart_orders:
                order_to_cart[order] = cart_no

        self.orders['CART_NO'] = self.orders['ORD_NO'].map(order_to_cart)

    def solve_picker_routing(self) -> None:
        self.orders = self.orders.sort_values(['CART_NO', 'LOC'])
        self.orders['SEQ'] = self.orders.groupby('CART_NO').cumcount() + 1

    def solve(self) -> pd.DataFrame:
        self.solve_storage_location()
        self.solve_order_batching()
        self.solve_picker_routing()
        return self.orders

def main(INPUT: pd.DataFrame, PARAMETER: pd.DataFrame, OD_MATRIX: pd.DataFrame) -> pd.DataFrame:
    solver = WarehouseSolver(INPUT, PARAMETER, OD_MATRIX)
    return solver.solve()
